# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

Query SQL
```sql
   {{ config(materialized='table') }}

SELECT extract(year from order_approved_at), count (order_id) as total
  FROM `dbt-shop-rodrigo-384318.dbt_shop.orders`
    where order_approved_at IS NOT NULL 
      group by extract(year from order_approved_at)

```   

``dbt run --models shop.orders_by_year``

![passo1](images/01-orders_by_year.png)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

Query SQL
```sql
SELECT customers.customer_state, count(orders.order_id) as total

 FROM `dbt-shop-rodrigo-384318.dbt_shop.orders` as orders 
  INNER JOIN `dbt_shop.customers` customers on orders.customer_id = customers.customer_id 
    group by customers.customer_state

```   

``dbt run --models shop.total_pedidos_por_estado``

![passo2](images/02-total_pedidos_por_estado.png)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

Query SQL
```sql
SELECT customers.customer_state, count(orders.order_id) as total

 FROM `dbt-shop-rodrigo-384318.dbt_shop.orders` as orders 
  INNER JOIN `dbt_shop.customers` customers on orders.customer_id = customers.customer_id    
     group by customers.customer_state
      having count(orders.order_id) > CAST('{{ var("number_of_orders") }}' as INT64)         

```   

``dbt run --models shop.total_pedidos_por_estado_qtd --vars 'number_of_orders: 5000'``

![passo3](images/03-number_of_orders.png)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

Query SQL
```sql
SELECT count (customer_id) as total_customers,
case
  --Norte
    when customer_state like 'AM' then 'Norte'
    when customer_state like 'RR' then 'Norte'
    when customer_state like 'AP' then 'Norte'
    when customer_state like 'PA' then 'Norte'
    when customer_state like 'TO' then 'Norte'
    when customer_state like 'RO' then 'Norte'
    when customer_state like 'AC' then 'Norte'

  --Nordeste
    when customer_state like 'RN' then 'Nordeste'
    when customer_state like 'CE' then 'Nordeste'
    when customer_state like 'MA' then 'Nordeste'
    when customer_state like 'PI' then 'Nordeste'
    when customer_state like 'PE' then 'Nordeste'
    when customer_state like 'PB' then 'Nordeste'
    when customer_state like 'SE' then 'Nordeste'
    when customer_state like 'AL' then 'Nordeste'
    when customer_state like 'BA' then 'Nordeste'

  -- Centro-Oeste
    when customer_state like 'MT' then 'Centro-Oeste'
    when customer_state like 'MS' then 'Centro-Oeste'
    when customer_state like 'GO' then 'Centro-Oeste'
    when customer_state like 'DF' then 'Centro-Oeste'

  --Sudeste
    when customer_state like 'SP' then 'Sudeste'
    when customer_state like 'RJ' then 'Sudeste'
    when customer_state like 'MG' then 'Sudeste'
    when customer_state like 'ES' then 'Sudeste'
  
  --Sul
  when customer_state like 'PR' then 'Sul'
  when customer_state like 'SC' then 'Sul'
  when customer_state like 'RS' then 'Sul'

  end as region


FROM `dbt-shop-rodrigo-384318.dbt_shop.customers` 
group by 
  case
  --Norte
    when customer_state like 'AM' then 'Norte'
    when customer_state like 'RR' then 'Norte'
    when customer_state like 'AP' then 'Norte'
    when customer_state like 'PA' then 'Norte'
    when customer_state like 'TO' then 'Norte'
    when customer_state like 'RO' then 'Norte'
    when customer_state like 'AC' then 'Norte'

  --Nordeste
    when customer_state like 'RN' then 'Nordeste'
    when customer_state like 'CE' then 'Nordeste'
    when customer_state like 'MA' then 'Nordeste'
    when customer_state like 'PI' then 'Nordeste'
    when customer_state like 'PE' then 'Nordeste'
    when customer_state like 'PB' then 'Nordeste'
    when customer_state like 'SE' then 'Nordeste'
    when customer_state like 'AL' then 'Nordeste'
    when customer_state like 'BA' then 'Nordeste'

  -- Centro-Oeste
    when customer_state like 'MT' then 'Centro-Oeste'
    when customer_state like 'MS' then 'Centro-Oeste'
    when customer_state like 'GO' then 'Centro-Oeste'
    when customer_state like 'DF' then 'Centro-Oeste'

  --Sudeste
    when customer_state like 'SP' then 'Sudeste'
    when customer_state like 'RJ' then 'Sudeste'
    when customer_state like 'MG' then 'Sudeste'
    when customer_state like 'ES' then 'Sudeste'
  
  --Sul
  when customer_state like 'PR' then 'Sul'
  when customer_state like 'SC' then 'Sul'
  when customer_state like 'RS' then 'Sul'
  end

``` 

``dbt run --models shop.customers_by_region``

![passo4](images/04-customers_by_region.png)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

Query SQL
```sql
{{ config(materialized='table') }}
SELECT *, (price + freight_value) as item_cost FROM `dbt-shop-rodrigo-384318.dbt_shop.order_items`  

```   

``dbt run --models shop.item_costs``

![passo5](images/05-item_costs.png)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

Query SQL
```sql
SELECT orders.order_id, orders.order_status, items.price, items.freight_value, 
  (items.price + items.freight_value) as total FROM `dbt-shop-rodrigo-384318.dbt_shop.orders` as orders
inner join `dbt_shop.order_items`items on items.order_id = orders.order_id

where order_status like "shipped"

```   

``dbt run --models shop.orders_shipped_costs``

![passo6](images/06-orders_shipped_costs.png)